In [10]:
!pip install konlpy



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## 감성, 일상 컬럼 분류 준지도 학습

* 초기 라벨링
    * token_list에 감성 또는 일상 키워드가 포함되어 있으면 감성=1 또는 일상=1로 지정
    * 포함되지 않으면 감성=None, 일상=None (즉, 라벨 미지정 상태)
* 모델 학습
    * 감성 또는 일상 라벨이 1로 지정된 샘플을 양성(positive) 샘플로 사용
    * 라벨이 None인 샘플들 중에서 무작위로 같은 개수만큼을 음성(negative) 샘플로 간주 (감성=0 또는 일상=0으로 설정)
    * 이렇게 구성된 양/음성 샘플로 TF-IDF 벡터화 → RandomForestClassifier 훈련
* 예측 대상
    * 라벨이 None이었던 샘플들 전체를 대상으로 TF-IDF 벡터화 후 예측
    * 예측값이 1인 경우, 해당 샘플의 라벨을 감성=1 또는 일상=1로 갱신 (예측값 0이면 0 넣음)

In [18]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import jpype
from collections import Counter

# JVM 경로 지정 (Mac 사용자용)
jvmpath = "/Users/byeon-ujung/Library/Java/JavaVirtualMachines/openjdk-23/Contents/Home/lib/server/libjvm.dylib"
okt = Okt(jvmpath=jvmpath)

# 1. 데이터 로딩 및 전처리
# CSV 파일에서 유튜브 영상 제목 데이터를 불러온다.
df = pd.read_csv("raw_data/youtube_info/병합후(후처리)/postP_all_youtubeAPI_info.csv")
df = df.dropna(subset=['title']).reset_index(drop=True)  # title 없는 행 제거

# 2. 텍스트 정제 함수 정의 (한글/영문/숫자 외 제거)
def clean_text(text):
    text = re.sub(r"[^\uAC00-\uD7A3a-zA-Z0-9\s]", "", str(text))
    return re.sub(r"\s+", " ", text).strip()

# 3. 형태소 분석 및 불용어 제거
custom_stopwords = {'playlist', 'Playlist', '리스트', '플레이', '모음', '음악', 'Collection'}
default_stopwords = {'은', '는', '이', '가', '을', '와', '과', '를', '에', '의', '도', '다', '한', '그리고', '으로', '부터', '에서'}
stopwords = default_stopwords.union(custom_stopwords)


# 감성 키워드
emotion_words = {
    '감성', '무드', '잔잔', '아늑', '몽환', '근사', '따뜻', '감미', '로맨틱', '인디', 'R&B', '새벽', '사랑', '혼자',
    '노을', '낭만', '여운', '포근', '섬세', '감동', '고요', '센치', '밤공기', '부드러운', '나른한', '그윽한', '온기',
    '멜로디', '여리여리', '감정선', '흘러가는', '그때 그 감성', '비', '잊지 못할', '쓸쓸한', '바람결', '피아노',
    '분위기', '감미로운', '여백', '밤하늘', '창가', '조용한', '빗소리', '꿈같은', '설레는', '따스한', '흐림',
    '스미는', '그날의', '목소리', '공기'
}

# 일상 키워드
daily_words = {
    '일상', '아침', '저녁', '출근', '퇴근', '산책', '브런치', '도심', '하루', '일기', '편안', '여유', '산뜻',
    '휴식', '오후', '오후3시', '자취', '혼밥', '책상', '창밖', '커피', '공원', '루틴', '나의 하루', '평온한',
    '나른한', '퇴근길', '혼자만의 시간', '작심삼일', '일요일', '카페', '째즈', '재즈', '등교', '하교',
    '루틴 음악', '아지트', '일상 회복', '무드등', '창문', '햇살', '방안', '자명종', '출근길', '하늘',
    '조용한 시간', '점심시간', '피로', '혼자 듣는', '느긋한', '거실', '여운'
}

df['token_list'] = df['title'].apply(lambda t: [tok for tok in Okt().morphs(clean_text(t)) if tok not in stopwords])

# 4. 감성/일상 키워드 정의
df['감성'] = df['token_list'].apply(lambda x: 1 if any(word in x for word in emotion_words) else None)
df['일상'] = df['token_list'].apply(lambda x: 1 if any(word in x for word in daily_words) else None)
df['감성_before'] = df['감성']
df['일상_before'] = df['일상']

# 5. 감성 모델 훈련
df_emotion_pos = df[df['감성'] == 1]  # 키워드 기반으로 감성=1로 라벨링된 데이터

df_emotion_neg_candidates = df[df['감성'].isna()]  # 라벨이 없는 나머지를 음성 후보로 본다.
df_emotion_neg = df_emotion_neg_candidates.sample(n=len(df_emotion_pos), random_state=42)
df_emotion_neg['감성'] = 0

df_emotion_train = pd.concat([df_emotion_pos, df_emotion_neg])

# ▶ TF-IDF 벡터화
# 각 token_list를 문자열로 변환해 문서로 보고, 전체 훈련 문서의 단어 빈도를 기반으로 TF-IDF 가중치를 부여함.
# 출력 X_emotion: (n_samples x n_features) 형태의 희소 행렬
vectorizer_emotion = TfidfVectorizer()
X_emotion = vectorizer_emotion.fit_transform(df_emotion_train['token_list'].apply(lambda x: " ".join(x)))
y_emotion = df_emotion_train['감성'].astype(int)

# ▶ 모델 훈련
# 훈련된 TF-IDF 벡터 X_emotion과 레이블 y_emotion(0/1)을 RandomForest 분류기에 학습시킴.
model_emotion = RandomForestClassifier(random_state=42)
model_emotion.fit(X_emotion, y_emotion)

# ▶ 예측 수행: 초기 라벨이 없던 샘플에 대해 감성 예측 수행
df_emotion_predict_target = df[df['감성'].isna()]
X_pred_emotion = vectorizer_emotion.transform(df_emotion_predict_target['token_list'].apply(lambda x: " ".join(x)))
pred_emotion = model_emotion.predict(X_pred_emotion)
df.loc[df_emotion_predict_target.index, '감성'] = pred_emotion

# 6. 일상 모델 훈련 (위와 동일 구조)
df_daily_pos = df[df['일상'] == 1]
df_daily_neg_candidates = df[df['일상'].isna()]
df_daily_neg = df_daily_neg_candidates.sample(n=len(df_daily_pos), random_state=42)
df_daily_neg['일상'] = 0

df_daily_train = pd.concat([df_daily_pos, df_daily_neg])

vectorizer_daily = TfidfVectorizer()
X_daily = vectorizer_daily.fit_transform(df_daily_train['token_list'].apply(lambda x: " ".join(x)))
y_daily = df_daily_train['일상'].astype(int)

model_daily = RandomForestClassifier(random_state=42)
model_daily.fit(X_daily, y_daily)

# 예측 수행
df_daily_predict_target = df[df['일상'].isna()]
X_pred_daily = vectorizer_daily.transform(df_daily_predict_target['token_list'].apply(lambda x: " ".join(x)))
pred_daily = model_daily.predict(X_pred_daily)
df.loc[df_daily_predict_target.index, '일상'] = pred_daily

# 7. 예측 결과 추적 및 출력
changed_emotion = df.loc[df_emotion_predict_target.index]
changed_emotion = changed_emotion[(changed_emotion['감성_before'].isna()) & (changed_emotion['감성'] == 1)]

changed_daily = df.loc[df_daily_predict_target.index]
changed_daily = changed_daily[(changed_daily['일상_before'].isna()) & (changed_daily['일상'] == 1)]

print("\n[감성] 예측으로 1로 바뀐 샘플:")
print(changed_emotion[['title', '감성_before', '감성']])

print("\n[일상] 예측으로 1로 바뀐 샘플:")
print(changed_daily[['title', '일상_before', '일상']])


[감성] 예측으로 1로 바뀐 샘플:
                                                   title  감성_before   감성
2      Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...        NaN  1.0
14     Playlist | 슬기로운 가을 나기 🍂 with 재즈 | 첫 소절만 들어도 마음...        NaN  1.0
20     Playlist | 재즈로 만나는 가을🚲 | 첫 소절만 들어도 기분이 좋아지는 신나...        NaN  1.0
60      𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 내 취향 저격한 음색 깡패 국내 여자 알앤비 r&amp;b 플레이리스트        NaN  1.0
545    [나플리 : Playlist🎧] 여름밤 듣기 좋은 힙한 노래 | JTBC 21031...        NaN  1.0
...                                                  ...        ...  ...
12936       Playlistㅣ편안한 휴식이 필요할 때 듣는 힐링음악 healing music        NaN  1.0
13354    [𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭] 첫 눈 오는 날, 따뜻한 초코라떼 한 잔❄｜하이라이트 플레이리스트        NaN  1.0
14108  𝒑𝒍𝒂𝒚𝒍𝒊𝒔𝒕. 눈이 오는 겨울밤❄ 듣기 좋은 잔잔한 발라드 플레이리스트🎧 l 비...        NaN  1.0
14258  12시간 루핑 BGM | 🏖 햇살 가득 수영장에서 듣기 좋은 🎷 여름 재즈 플레이리...        NaN  1.0
14273  𝙅𝙖𝙯𝙯 &amp; 𝘽𝙤𝙨𝙨𝙖 | 🏖 햇살 가득 수영장에서 듣기 좋은 🎷 여름 재즈...        NaN  1.0

[90 rows x 3 columns]

[일상] 예측으로 1로 바뀐 샘플:
                                                   title  일

In [19]:
tmp = changed_emotion.loc[:, ['title','token_list', '감성_before', '감성']]
tmp

,title,token_list,감성_before,감성
2,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"[여유롭게, 쉬, 고, 싶을, 때, 잔잔히, 틀기, 좋은, 아늑한, 겨울, 재즈, ...",NaN,1.0
14,Playlist | 슬기로운 가을 나기 🍂 with 재즈 | 첫 소절만 들어도 마음...,"[슬기로운, 가을, 나기, with, 재즈, 첫, 소, 절, 만, 들어도, 마음, ...",NaN,1.0
20,Playlist | 재즈로 만나는 가을🚲 | 첫 소절만 들어도 기분이 좋아지는 신나...,"[재즈, 로, 만나는, 가을, 첫, 소, 절, 만, 들어도, 기분, 좋아지는, 신나...",NaN,1.0
60,𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 내 취향 저격한 음색 깡패 국내 여자 알앤비 r&amp;b 플레이리스트,"[내, 취향, 저격, 음색, 깡패, 국내, 여자, 알앤비, rampb]",NaN,1.0
545,[나플리 : Playlist🎧] 여름밤 듣기 좋은 힙한 노래 | JTBC 21031...,"[나, 플리, 여름밤, 듣기, 좋은, 힙, 노래, JTBC, 210315, 방송, 외]",NaN,1.0
...,...,...,...,...
12936,Playlistㅣ편안한 휴식이 필요할 때 듣는 힐링음악 healing music,"[편안한, 휴식, 필요할, 때, 듣는, 힐링, healing, music]",NaN,1.0
13354,"[𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭] 첫 눈 오는 날, 따뜻한 초코라떼 한 잔❄｜하이라이트 플레이리스트","[첫, 눈, 오는, 날, 따뜻한, 초코, 라떼, 잔, 하이라이트]",NaN,1.0
14108,𝒑𝒍𝒂𝒚𝒍𝒊𝒔𝒕. 눈이 오는 겨울밤❄ 듣기 좋은 잔잔한 발라드 플레이리스트🎧 l 비...,"[눈, 오는, 겨울, 밤, 듣기, 좋은, 잔잔한, 발라드, l, 비긴, 어게인, 오...",NaN,1.0
14258,12시간 루핑 BGM | 🏖 햇살 가득 수영장에서 듣기 좋은 🎷 여름 재즈 플레이리...,"[12시간, 루핑, BGM, 햇살, 가득, 수영장, 듣기, 좋은, 여름, 재즈, 휴...",NaN,1.0


In [24]:
# 예측으로 1로 바뀐 감성 샘플 중 하나 선택
example_row = changed_emotion.iloc[0]
example_title = example_row['title']
example_tokens = example_row['token_list']
example_text = " ".join(example_tokens)

# 해당 샘플의 TF-IDF 벡터 추출
example_vector = vectorizer_emotion.transform([example_text])
feature_names = vectorizer_emotion.get_feature_names_out()

# (index, tfidf값) 추출
dense_vec = example_vector.todense().tolist()[0]
top_indices = np.argsort(dense_vec)[::-1][:10]

# 상위 TF-IDF 단어 정보 정리
top_words_info = [
    {
        '단어': feature_names[i],
        'TF-IDF': dense_vec[i],
        'TF': example_tokens.count(feature_names[i]),
        'DF': vectorizer_emotion.idf_[i]
    }
    for i in top_indices if dense_vec[i] > 0
]

tf_idf_result = pd.DataFrame(top_words_info)
tf_idf_result

,단어,TF-IDF,TF,DF
0,틀기,0.418544,1,8.992832
1,여유롭게,0.386283,1,8.299685
2,잔잔히,0.379109,1,8.145534
3,아늑한,0.358073,1,7.693549
4,background,0.294622,0,6.330244
5,cozy,0.274681,0,5.901789
6,싶을,0.260245,1,5.591635
7,relaxing,0.199493,0,4.286309
8,music,0.173528,0,3.728416
9,jazz,0.173208,0,3.721543


In [26]:
example_title

'Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈\xa0| Cozy Jazz | Relaxing Background Music💓'

In [28]:
example_tokens

['여유롭게',
 '쉬',
 '고',
 '싶을',
 '때',
 '잔잔히',
 '틀기',
 '좋은',
 '아늑한',
 '겨울',
 '재즈',
 'Cozy',
 'Jazz',
 'Relaxing',
 'Background',
 'Music']

In [30]:
tmp.to_csv('감성1로바뀐것.csv', encoding='utf-8-sig')
tf_idf_result.to_csv('TF-IDF_.csv', encoding='utf-8-sig')

In [7]:
changed_emotion[['title', '감성_before', '감성']]

,title,감성_before,감성
2,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,NaN,1.0
14,Playlist | 슬기로운 가을 나기 🍂 with 재즈 | 첫 소절만 들어도 마음...,NaN,1.0
20,Playlist | 재즈로 만나는 가을🚲 | 첫 소절만 들어도 기분이 좋아지는 신나...,NaN,1.0
60,𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 내 취향 저격한 음색 깡패 국내 여자 알앤비 r&amp;b 플레이리스트,NaN,1.0
545,[나플리 : Playlist🎧] 여름밤 듣기 좋은 힙한 노래 | JTBC 21031...,NaN,1.0
...,...,...,...
12936,Playlistㅣ편안한 휴식이 필요할 때 듣는 힐링음악 healing music,NaN,1.0
13354,"[𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭] 첫 눈 오는 날, 따뜻한 초코라떼 한 잔❄｜하이라이트 플레이리스트",NaN,1.0
14108,𝒑𝒍𝒂𝒚𝒍𝒊𝒔𝒕. 눈이 오는 겨울밤❄ 듣기 좋은 잔잔한 발라드 플레이리스트🎧 l 비...,NaN,1.0
14258,12시간 루핑 BGM | 🏖 햇살 가득 수영장에서 듣기 좋은 🎷 여름 재즈 플레이리...,NaN,1.0


In [8]:
changed_daily[['title', '일상_before', '일상']]

,title,일상_before,일상
200,"[Playlist] 감미로운 보사노바 선율, 조빔과 함께 | Bossa Nova ...",NaN,1.0
736,[Playlist] 내가 주말을 사랑하는 방식 | soft &amp; relaxin...,NaN,1.0
907,[Playlist] ⛅️ 나만 알고 있기엔 너무 아까운 노래들 l 플레이리스트 감성...,NaN,1.0
928,[Playlist] 첫 곡 듣고 바로 프뮤했음. 👍❤️‍🔥🌸내가 사랑하는 띵곡 노래...,NaN,1.0
1087,𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 그냥 아무것도 안하고 Chill 하고 싶을 때 🎶 | 완벽한 힐링 ...,NaN,1.0
...,...,...,...
14134,𝑷𝒍𝒂𝒚𝒍𝒊𝒔𝒕 몽글몽글 디즈니 깔 좋아해..?ㅣJazz/ Chill Piano플레...,NaN,1.0
14266,𝙋𝙞𝙖𝙣𝙤 | 🎂 𝑯𝒂𝒑𝒑𝒚 𝑩𝒊𝒓𝒕𝒉𝒅𝒂𝒚 | 🍒 빈티지 체리 케이크 한 조각 그...,NaN,1.0
14296,𝙥𝙡𝙖𝙮𝙡𝙞𝙨𝙩🐶 감성 넘치는 소품샵에서 흘러나오는 플레이리스트🫧ㅣ가사 없는 음악ㅣ...,NaN,1.0
14309,"𝙥𝙡𝙖𝙮𝙡𝙞𝙨𝙩🏄 빈티지 바이브 감각적인 플레이리스트ㅣ가사 없는 음악ㅣlofi, ...",NaN,1.0


In [9]:
df.loc[df['감성']==1, : ]

,Unnamed: 0,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상,감성_before,일상_before
0,0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"[재즈, 로, 무드있, 고, 잔잔하게, 내, 방, 근사하게, 만드는, 로맨틱, 분위...",1.0,1.0,1.0,1.0
2,2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"[여유롭게, 쉬, 고, 싶을, 때, 잔잔히, 틀기, 좋은, 아늑한, 겨울, 재즈, ...",1.0,1.0,NaN,1.0
4,4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","[재즈, 내리는, 계절, 겨울, 눈, 오는, 날, 듣기, 좋은, 마음, 저절로, 따...",1.0,1.0,1.0,1.0
5,5,Jazzne | 기분Jazz네,287000,Playlist | 적당히 신나는 템포의 편안한 겨울 감성 재즈 with 뉴욕🗽🇺🇸...,https://www.youtube.com/watch?v=xF_r6vfY9L4,2025-01-04T23:50:00Z,https://i.ytimg.com/vi/xF_r6vfY9L4/hqdefault.jpg,158522,789,29,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",xF_r6vfY9L4,Playlist | 적당히 신나는 템포의 편안한 겨울 감성 재즈 with 뉴욕🗽🇺🇸...,"[적당히, 신나는, 템포, 편안한, 겨울, 감성, 재즈, with, 뉴욕, 뉴욕, ...",1.0,1.0,1.0,1.0
6,6,Jazzne | 기분Jazz네,287000,"Playlist | 눈 내리는 겨울, 재즈와 함께 ❤️🎧 | 틀어놓고 편안히 듣는 ...",https://www.youtube.com/watch?v=QJuQNPje0Y4,2024-12-28T23:50:00Z,https://i.ytimg.com/vi/QJuQNPje0Y4/hqdefault.jpg,299456,1157,26,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",QJuQNPje0Y4,"Playlist | 눈 내리는 겨울, 재즈와 함께 ❤️🎧 | 틀어놓고 편안히 듣는 ...","[눈, 내리는, 겨울, 재즈, 함께, 틀어놓고, 편안히, 듣는, 부드러운, 감성, ...",1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14327,14327,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 신주쿠 중앙공원에서 즐기는 싱그러운 시작의 멜로디🍃 ㅣ ...,https://www.youtube.com/watch?v=AbDJl2CTPLs,2024-04-23T01:23:38Z,https://i.ytimg.com/vi/AbDJl2CTPLs/hqdefault.jpg,39438,512,24,NaN,AbDJl2CTPLs,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 신주쿠 중앙공원에서 즐기는 싱그러운 시작의 멜로디🍃 ㅣ ...,"[신주쿠, 중앙, 공원, 즐기는, 싱그러운, 시작, 멜로디, 청량하고, 상쾌한, 제...",1.0,1.0,1.0,1.0
14333,14333,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,https://www.youtube.com/watch?v=Sh41rnVFlhU,2024-03-09T09:23:50Z,https://i.ytimg.com/vi/Sh41rnVFlhU/hqdefault.jpg,7418,98,18,NaN,Sh41rnVFlhU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,"[도쿄, 긴자, 밤, 닮은, 중독, 적, 인, 리듬, 멜로디, 힙, 사운드, 제이팝...",1.0,0.0,1.0,NaN
14334,14334,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,https://www.youtube.com/watch?v=z1UGEkHIoCY,2024-02-26T10:00:40Z,https://i.ytimg.com/vi/z1UGEkHIoCY/hqdefault.jpg,16421,218,9,NaN,z1UGEkHIoCY,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,"[쓸쓸한, 도쿄, 퇴근길, 조용한, 밤, 듣기, 좋은, 제이팝, JPOP]",1.0,1.0,1.0,1.0
14335,14335,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,https://www.youtube.com/watch?v=xVaTzgcJttI,2024-02-22T04:03:09Z,https://i.ytimg.com/vi/xVaTzgcJttI/hqdefault.jpg,12654,122,9,NaN,xVaTzgcJttI,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,"[사랑, 하는, 너, 열차, 안, 함께, 노래, 듣고싶어, 애정, 싹트는, 제이팝,...",1.0,0.0,1.0,NaN


In [10]:
df.loc[df['일상']==1, : ]

,Unnamed: 0,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상,감성_before,일상_before
0,0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"[재즈, 로, 무드있, 고, 잔잔하게, 내, 방, 근사하게, 만드는, 로맨틱, 분위...",1.0,1.0,1.0,1.0
1,1,Jazzne | 기분Jazz네,287000,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,https://www.youtube.com/watch?v=kERWAGtltBU,2025-02-01T23:50:00Z,https://i.ytimg.com/vi/kERWAGtltBU/hqdefault.jpg,80929,404,22,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",kERWAGtltBU,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,"[일, 할, 때, 틀어, 놓기, 좋은, 업무, 용, 재즈, 집중, 력, 높이는, 적...",0.0,1.0,NaN,1.0
2,2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"[여유롭게, 쉬, 고, 싶을, 때, 잔잔히, 틀기, 좋은, 아늑한, 겨울, 재즈, ...",1.0,1.0,NaN,1.0
3,3,Jazzne | 기분Jazz네,287000,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,https://www.youtube.com/watch?v=CI331OcJ8Do,2025-01-18T23:50:00Z,https://i.ytimg.com/vi/CI331OcJ8Do/hqdefault.jpg,265932,1042,45,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",CI331OcJ8Do,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,"[기분, 좋은, 아침, 여, 모닝, 겨울, 재즈, 하루, 시작, 하며, 듣기, 좋은...",0.0,1.0,NaN,1.0
4,4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","[재즈, 내리는, 계절, 겨울, 눈, 오는, 날, 듣기, 좋은, 마음, 저절로, 따...",1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14327,14327,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 신주쿠 중앙공원에서 즐기는 싱그러운 시작의 멜로디🍃 ㅣ ...,https://www.youtube.com/watch?v=AbDJl2CTPLs,2024-04-23T01:23:38Z,https://i.ytimg.com/vi/AbDJl2CTPLs/hqdefault.jpg,39438,512,24,NaN,AbDJl2CTPLs,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 신주쿠 중앙공원에서 즐기는 싱그러운 시작의 멜로디🍃 ㅣ ...,"[신주쿠, 중앙, 공원, 즐기는, 싱그러운, 시작, 멜로디, 청량하고, 상쾌한, 제...",1.0,1.0,1.0,1.0
14330,14330,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 오키나와 잔탄 공원의 레스토랑에서 바다를 바라보며🌊ㅣ 여...,https://www.youtube.com/watch?v=b2CdWAFQkIQ,2024-03-28T03:30:12Z,https://i.ytimg.com/vi/b2CdWAFQkIQ/hqdefault.jpg,28175,336,27,NaN,b2CdWAFQkIQ,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 오키나와 잔탄 공원의 레스토랑에서 바다를 바라보며🌊ㅣ 여...,"[오키나와, 잔, 탄, 공원, 레스토랑, 바다, 바라보며, 여유, 느껴지는, 제이팝...",0.0,1.0,NaN,1.0
14331,14331,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,https://www.youtube.com/watch?v=fLNAk4NYOJU,2024-03-21T04:00:19Z,https://i.ytimg.com/vi/fLNAk4NYOJU/hqdefault.jpg,64905,790,27,NaN,fLNAk4NYOJU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,"[벚꽃, 만, 개, 하, 봄, 봄, 햇살, 함께, 듣기, 좋은, 제이팝, JPOP]",0.0,1.0,NaN,1.0
14334,14334,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,https://www.youtube.com/watch?v=z1UGEkHIoCY,2024-02-26T10:00:40Z,https://i.ytimg.com/vi/z1UGEkHIoCY/hqdefault.jpg,16421,218,9,NaN,z1UGEkHIoCY,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,"[쓸쓸한, 도쿄, 퇴근길, 조용한, 밤, 듣기, 좋은, 제이팝, JPOP]",1.0,1.0,1.0,1.0


In [11]:
# 이걸로 그냥 사용하는 것은 어떨지..?
df.loc[(df['감성']==1) | (df['일상']==1), : ]

,Unnamed: 0,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상,감성_before,일상_before
0,0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"[재즈, 로, 무드있, 고, 잔잔하게, 내, 방, 근사하게, 만드는, 로맨틱, 분위...",1.0,1.0,1.0,1.0
1,1,Jazzne | 기분Jazz네,287000,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,https://www.youtube.com/watch?v=kERWAGtltBU,2025-02-01T23:50:00Z,https://i.ytimg.com/vi/kERWAGtltBU/hqdefault.jpg,80929,404,22,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",kERWAGtltBU,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,"[일, 할, 때, 틀어, 놓기, 좋은, 업무, 용, 재즈, 집중, 력, 높이는, 적...",0.0,1.0,NaN,1.0
2,2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"[여유롭게, 쉬, 고, 싶을, 때, 잔잔히, 틀기, 좋은, 아늑한, 겨울, 재즈, ...",1.0,1.0,NaN,1.0
3,3,Jazzne | 기분Jazz네,287000,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,https://www.youtube.com/watch?v=CI331OcJ8Do,2025-01-18T23:50:00Z,https://i.ytimg.com/vi/CI331OcJ8Do/hqdefault.jpg,265932,1042,45,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",CI331OcJ8Do,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,"[기분, 좋은, 아침, 여, 모닝, 겨울, 재즈, 하루, 시작, 하며, 듣기, 좋은...",0.0,1.0,NaN,1.0
4,4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","[재즈, 내리는, 계절, 겨울, 눈, 오는, 날, 듣기, 좋은, 마음, 저절로, 따...",1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14331,14331,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,https://www.youtube.com/watch?v=fLNAk4NYOJU,2024-03-21T04:00:19Z,https://i.ytimg.com/vi/fLNAk4NYOJU/hqdefault.jpg,64905,790,27,NaN,fLNAk4NYOJU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,"[벚꽃, 만, 개, 하, 봄, 봄, 햇살, 함께, 듣기, 좋은, 제이팝, JPOP]",0.0,1.0,NaN,1.0
14333,14333,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,https://www.youtube.com/watch?v=Sh41rnVFlhU,2024-03-09T09:23:50Z,https://i.ytimg.com/vi/Sh41rnVFlhU/hqdefault.jpg,7418,98,18,NaN,Sh41rnVFlhU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,"[도쿄, 긴자, 밤, 닮은, 중독, 적, 인, 리듬, 멜로디, 힙, 사운드, 제이팝...",1.0,0.0,1.0,NaN
14334,14334,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,https://www.youtube.com/watch?v=z1UGEkHIoCY,2024-02-26T10:00:40Z,https://i.ytimg.com/vi/z1UGEkHIoCY/hqdefault.jpg,16421,218,9,NaN,z1UGEkHIoCY,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,"[쓸쓸한, 도쿄, 퇴근길, 조용한, 밤, 듣기, 좋은, 제이팝, JPOP]",1.0,1.0,1.0,1.0
14335,14335,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,https://www.youtube.com/watch?v=xVaTzgcJttI,2024-02-22T04:03:09Z,https://i.ytimg.com/vi/xVaTzgcJttI/hqdefault.jpg,12654,122,9,NaN,xVaTzgcJttI,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,"[사랑, 하는, 너, 열차, 안, 함께, 노래, 듣고싶어, 애정, 싹트는, 제이팝,...",1.0,0.0,1.0,NaN


In [12]:
from collections import Counter

# 기존 감성 키워드
emotion_words = {
    '감성', '무드', '잔잔', '아늑', '몽환', '근사', '따뜻', '감미', '로맨틱', '인디', 'R&B', '새벽', '사랑', '혼자',
    '노을', '낭만', '여운', '포근', '섬세', '감동', '고요', '센치', '밤공기', '부드러운', '나른한', '그윽한', '온기',
    '멜로디', '여리여리', '감정선', '흘러가는', '그때 그 감성', '비', '잊지 못할', '쓸쓸한', '바람결', '피아노',
    '분위기', '감미로운', '여백', '밤하늘', '창가', '조용한', '빗소리', '꿈같은', '설레는', '따스한', '흐림',
    '스미는', '그날의', '목소리', '공기'
}

# 기존 일상 키워드
daily_words = {
    '일상', '아침', '저녁', '출근', '퇴근', '산책', '브런치', '도심', '하루', '일기', '편안', '여유', '산뜻',
    '휴식', '오후', '오후3시', '자취', '혼밥', '책상', '창밖', '커피', '공원', '루틴', '나의 하루', '평온한',
    '나른한', '퇴근길', '혼자만의 시간', '작심삼일', '일요일', '카페', '째즈', '재즈', '등교', '하교',
    '루틴 음악', '아지트', '일상 회복', '무드등', '창문', '햇살', '방안', '자명종', '출근길', '하늘',
    '조용한 시간', '점심시간', '피로', '혼자 듣는', '느긋한', '거실', '여운'
}

# --- 감성 ---
changed_emotion = df.loc[df_emotion_predict_target.index]
changed_emotion = changed_emotion[(changed_emotion['감성_before'].isna()) & (changed_emotion['감성'] == 1)]

tokens_emotion = [token for row in changed_emotion['token_list'] for token in row]
new_emotion_tokens = [token for token in tokens_emotion if token not in emotion_words]
top_emotion = Counter(new_emotion_tokens).most_common(20)

print("\n✨ 감성=1로 예측된 샘플 중 기존 키워드에 없던 상위 단어:")
for word, count in top_emotion:
    print(f"{word}: {count}회")

# --- 일상 ---
changed_daily = df.loc[df_daily_predict_target.index]
changed_daily = changed_daily[(changed_daily['일상_before'].isna()) & (changed_daily['일상'] == 1)]

tokens_daily = [token for row in changed_daily['token_list'] for token in row]
new_daily_tokens = [token for token in tokens_daily if token not in daily_words]
top_daily = Counter(new_daily_tokens).most_common(20)

print("\n✨ 일상=1로 예측된 샘플 중 기존 키워드에 없던 상위 단어:")
for word, count in top_daily:
    print(f"{word}: {count}회")



✨ 감성=1로 예측된 샘플 중 기존 키워드에 없던 상위 단어:
재즈: 47회
좋은: 25회
가을: 24회
듣기: 22회
amp: 20회
힐링: 19회
겨울: 18회
눈: 16회
오는: 15회
Jazz: 14회
노래: 14회
날: 13회
발라드: 13회
때: 11회
JAZZ: 11회
위: 11회
마음: 10회
아침: 10회
휴식: 10회
Piano: 10회

✨ 일상=1로 예측된 샘플 중 기존 키워드에 없던 상위 단어:
Piano: 144회
겨울: 139회
피아노: 139회
amp: 136회
감성: 110회
좋은: 92회
플리: 86회
힐링: 85회
연주: 83회
Relaxing: 79회
Music: 76회
가을: 69회
팝: 67회
듣기: 66회
Winter: 62회
Groove: 62회
Jazz: 61회
그루브: 59회
기분: 58회
하는: 57회


* 감성 1, 일상 1에 자주 등장하는 단어에 필요없는 [Playlist, 리스트, 플레이, playlist, 모음, 음악, Collection] 이 있어서 맨 위의 코드도 수정함

### 감성 '또는' 일상인 영상들을 가져오는 것이 어떨지... (감성과 일상 관련 영상들을 모두 가져옴)

In [22]:
# 조건을 만족하는 행 필터링
filtered_df = df.loc[(df['감성'] == 1) | (df['일상'] == 1), :]

# 감성_before, 일상_before 컬럼 제외
filtered_df = filtered_df.drop(columns=['Unnamed: 0', '감성_before', '일상_before'])
filtered_df.reset_index(drop=True, inplace=True)
filtered_df.index.name = 'video_idx'

# CSV 저장
filtered_df.to_csv("categ_all_youtubeAPI_info.csv", index=False, encoding='utf-8-sig')

In [24]:
filtered_df

,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상
video_idx,,,,,,,,,,,,,,,
0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"[재즈, 로, 무드있, 고, 잔잔하게, 내, 방, 근사하게, 만드는, 로맨틱, 분위...",1.0,1.0
1,Jazzne | 기분Jazz네,287000,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,https://www.youtube.com/watch?v=kERWAGtltBU,2025-02-01T23:50:00Z,https://i.ytimg.com/vi/kERWAGtltBU/hqdefault.jpg,80929,404,22,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",kERWAGtltBU,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,"[일, 할, 때, 틀어, 놓기, 좋은, 업무, 용, 재즈, 집중, 력, 높이는, 적...",0.0,1.0
2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"[여유롭게, 쉬, 고, 싶을, 때, 잔잔히, 틀기, 좋은, 아늑한, 겨울, 재즈, ...",1.0,1.0
3,Jazzne | 기분Jazz네,287000,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,https://www.youtube.com/watch?v=CI331OcJ8Do,2025-01-18T23:50:00Z,https://i.ytimg.com/vi/CI331OcJ8Do/hqdefault.jpg,265932,1042,45,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",CI331OcJ8Do,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,"[기분, 좋은, 아침, 여, 모닝, 겨울, 재즈, 하루, 시작, 하며, 듣기, 좋은...",0.0,1.0
4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","[재즈, 내리는, 계절, 겨울, 눈, 오는, 날, 듣기, 좋은, 마음, 저절로, 따...",1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6409,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,https://www.youtube.com/watch?v=fLNAk4NYOJU,2024-03-21T04:00:19Z,https://i.ytimg.com/vi/fLNAk4NYOJU/hqdefault.jpg,64905,790,27,NaN,fLNAk4NYOJU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,"[벚꽃, 만, 개, 하, 봄, 봄, 햇살, 함께, 듣기, 좋은, 제이팝, JPOP]",0.0,1.0
6410,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,https://www.youtube.com/watch?v=Sh41rnVFlhU,2024-03-09T09:23:50Z,https://i.ytimg.com/vi/Sh41rnVFlhU/hqdefault.jpg,7418,98,18,NaN,Sh41rnVFlhU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,"[도쿄, 긴자, 밤, 닮은, 중독, 적, 인, 리듬, 멜로디, 힙, 사운드, 제이팝...",1.0,0.0
6411,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,https://www.youtube.com/watch?v=z1UGEkHIoCY,2024-02-26T10:00:40Z,https://i.ytimg.com/vi/z1UGEkHIoCY/hqdefault.jpg,16421,218,9,NaN,z1UGEkHIoCY,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,"[쓸쓸한, 도쿄, 퇴근길, 조용한, 밤, 듣기, 좋은, 제이팝, JPOP]",1.0,1.0


### 아래 수정 전임

In [58]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import jpype

# JVM 경로 지정 (Mac 사용자용)
jvmpath = "/Users/byeon-ujung/Library/Java/JavaVirtualMachines/openjdk-23/Contents/Home/lib/server/libjvm.dylib"
okt = Okt(jvmpath=jvmpath)

# 데이터 로딩
df = pd.read_csv("postP_all_youtubeAPI_info.csv")
df = df.dropna(subset=['title']).reset_index(drop=True)

# 텍스트 정제
def clean_text(text):
    text = re.sub(r"[^\uAC00-\uD7A3a-zA-Z0-9\s]", "", str(text))
    return re.sub(r"\s+", " ", text).strip()

# 형태소 분석 및 불용어 제거
stopwords = {'은', '는', '이', '가', '을', '와', '과', '를', '에', '의', '도', '다', '한', '그리고', '으로', '부터'}
token_lists = []
for title in df['title']:
    cleaned = clean_text(title)
    tokens = okt.morphs(cleaned)
    tokens = [t for t in tokens if t not in stopwords]
    token_lists.append(tokens)
df['token_list'] = token_lists

# 키워드 사전 정의
emotion_words = {
    '감성', '무드', '잔잔', '아늑', '몽환', '근사', '따뜻', '감미', '로맨틱', '인디', 'R&B', '새벽', '사랑', '혼자',
    '노을', '낭만', '여운', '포근', '섬세', '감동', '고요', '센치', '밤공기', '부드러운', '나른한', '그윽한', '온기',
    '멜로디', '여리여리', '감정선', '흘러가는', '그때 그 감성', '비', '잊지 못할', '쓸쓸한', '바람결', '피아노',
    '분위기', '감미로운', '여백', '밤하늘', '창가', '조용한', '빗소리', '꿈같은', '설레는', '따스한', '흐림',
    '스미는', '그날의', '목소리', '공기'
}
daily_words = {
    '일상', '아침', '저녁', '출근', '퇴근', '산책', '브런치', '도심', '하루', '일기', '편안', '여유', '산뜻',
    '휴식', '오후', '오후3시', '자취', '혼밥', '책상', '창밖', '커피', '공원', '루틴', '나의 하루', '평온한',
    '나른한', '퇴근길', '혼자만의 시간', '작심삼일', '일요일', '카페', '째즈', '재즈', '등교', '하교',
    '루틴 음악', '아지트', '일상 회복', '무드등', '창문', '햇살', '방안', '자명종', '출근길', '하늘',
    '조용한 시간', '점심시간', '피로', '혼자 듣는', '느긋한', '거실', '여운'
}

# 초기 감성/일상 라벨링
df['감성'] = df['token_list'].apply(lambda x: 1 if any(word in x for word in emotion_words) else None)
df['일상'] = df['token_list'].apply(lambda x: 1 if any(word in x for word in daily_words) else None)

# ---------------------- 감성 라벨링 및 학습 ----------------------
# 감성 라벨링된 데이터
df_emotion_pos = df[df['감성'] == 1].copy()
df_emotion_neg_candidates = df[df['감성'].isna()].copy()

# 감성 음성 샘플 무작위 추출 (양성 샘플 수만큼)
df_emotion_neg = df_emotion_neg_candidates.sample(n=len(df_emotion_pos), random_state=42).copy()
df_emotion_neg['감성'] = 0

# 감성 훈련 데이터 결합
df_emotion_train = pd.concat([df_emotion_pos, df_emotion_neg])
vectorizer_emotion = TfidfVectorizer()
X_emotion = vectorizer_emotion.fit_transform(df_emotion_train['token_list'].apply(lambda x: " ".join(x)))
y_emotion = df_emotion_train['감성'].astype(int)

model_emotion = RandomForestClassifier(random_state=42)
model_emotion.fit(X_emotion, y_emotion)

# 예측 대상: 원래 라벨이 비어있던 감성 샘플
df_emotion_predict_target = df[df['감성'].isna()].copy()
X_pred_emotion = vectorizer_emotion.transform(df_emotion_predict_target['token_list'].apply(lambda x: " ".join(x)))
pred_emotion = model_emotion.predict(X_pred_emotion)
df.loc[df_emotion_predict_target.index, '감성'] = pred_emotion

# ---------------------- 일상 라벨링 및 학습 ----------------------
# 일상 라벨링된 데이터
df_daily_pos = df[df['일상'] == 1].copy()
df_daily_neg_candidates = df[df['일상'].isna()].copy()

# 일상 음성 샘플 무작위 추출
df_daily_neg = df_daily_neg_candidates.sample(n=len(df_daily_pos), random_state=42).copy()
df_daily_neg['일상'] = 0

# 일상 훈련 데이터 결합
df_daily_train = pd.concat([df_daily_pos, df_daily_neg])
vectorizer_daily = TfidfVectorizer()
X_daily = vectorizer_daily.fit_transform(df_daily_train['token_list'].apply(lambda x: " ".join(x)))
y_daily = df_daily_train['일상'].astype(int)

model_daily = RandomForestClassifier(random_state=42)
model_daily.fit(X_daily, y_daily)

# 예측 대상
df_daily_predict_target = df[df['일상'].isna()].copy()
X_pred_daily = vectorizer_daily.transform(df_daily_predict_target['token_list'].apply(lambda x: " ".join(x)))
pred_daily = model_daily.predict(X_pred_daily)
df.loc[df_daily_predict_target.index, '일상'] = pred_daily

# ---------------------- 결과 ----------------------
print("감성 라벨 분포:\n", df['감성'].value_counts())
print("일상 라벨 분포:\n", df['일상'].value_counts())
df[['title', '감성', '일상']]


감성 라벨 분포:
 감성
0.0    9764
1.0    4576
Name: count, dtype: int64
일상 라벨 분포:
 일상
0.0    10916
1.0     3424
Name: count, dtype: int64


,title,감성,일상
0,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,1.0,1.0
1,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,0.0,1.0
2,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,0.0,1.0
3,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,1.0,1.0
4,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",1.0,1.0
...,...,...,...
14335,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,1.0,0.0
14336,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 비 오는 오후에 도쿄 메이지 신궁에서☔ ㅣ 비 오는 날에...,1.0,1.0
14337,[𝐒𝐕𝐓 𝐏𝐋𝐀𝐘𝐋𝐈𝐒𝐓] 나를 토닥토닥 위로해주는 세븐틴 노래 모음⋆˚ ☁︎ ˚⋆...,0.0,0.0
14338,[𝐒𝐕𝐓 𝐏𝐋𝐀𝐘𝐋𝐈𝐒𝐓] 나를 토닥토닥 위로해주는 세븐틴 노래 모음⋆˚ ☁︎ ˚⋆...,0.0,0.0


In [73]:
df.loc[(df['감성']==1) | (df['일상']==1), : ]

,Unnamed: 0,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상
0,0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"[Playlist, 재즈, 로, 무드있, 고, 잔잔하게, 내, 방, 근사하게, 만드...",1.0,1.0
1,1,Jazzne | 기분Jazz네,287000,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,https://www.youtube.com/watch?v=kERWAGtltBU,2025-02-01T23:50:00Z,https://i.ytimg.com/vi/kERWAGtltBU/hqdefault.jpg,80929,404,22,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",kERWAGtltBU,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,"[Playlist, 일, 할, 때, 틀어, 놓기, 좋은, 업무, 용, 재즈, 집중,...",0.0,1.0
2,2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"[Playlist, 여유롭게, 쉬, 고, 싶을, 때, 잔잔히, 틀기, 좋은, 아늑한...",0.0,1.0
3,3,Jazzne | 기분Jazz네,287000,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,https://www.youtube.com/watch?v=CI331OcJ8Do,2025-01-18T23:50:00Z,https://i.ytimg.com/vi/CI331OcJ8Do/hqdefault.jpg,265932,1042,45,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",CI331OcJ8Do,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,"[Playlist, 기분, 좋은, 아침, 여, 모닝, 겨울, 재즈, 하루, 시작, ...",1.0,1.0
4,4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","[Playlist, 재즈, 내리는, 계절, 겨울, 눈, 오는, 날, 듣기, 좋은, ...",1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14332,14332,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 하네다 공항에서 도쿄로 향하는 밤🚘 ㅣ 드라이브 할 때 ...,https://www.youtube.com/watch?v=t6TqF6HS0TA,2024-03-13T13:08:35Z,https://i.ytimg.com/vi/t6TqF6HS0TA/hqdefault.jpg,118207,1371,25,NaN,t6TqF6HS0TA,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 하네다 공항에서 도쿄로 향하는 밤🚘 ㅣ 드라이브 할 때 ...,"[하네다, 공항, 에서, 도쿄, 로, 향, 하는, 밤, 드라이브, 할, 때, 듣기,...",0.0,1.0
14333,14333,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,https://www.youtube.com/watch?v=Sh41rnVFlhU,2024-03-09T09:23:50Z,https://i.ytimg.com/vi/Sh41rnVFlhU/hqdefault.jpg,7418,98,18,NaN,Sh41rnVFlhU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,"[도쿄, 긴자, 밤, 닮은, 중독, 적, 인, 리듬, 멜로디, 힙, 사운드, 제이팝...",1.0,0.0
14334,14334,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,https://www.youtube.com/watch?v=z1UGEkHIoCY,2024-02-26T10:00:40Z,https://i.ytimg.com/vi/z1UGEkHIoCY/hqdefault.jpg,16421,218,9,NaN,z1UGEkHIoCY,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,"[쓸쓸한, 도쿄, 퇴근길, 조용한, 밤, 듣기, 좋은, 제이팝, JPOP, 플레이,...",1.0,1.0
14335,14335,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,https://www.youtube.com/watch?v=xVaTzgcJttI,2024-02-22T04:03:09Z,https://i.ytimg.com/vi/xVaTzgcJttI/hqdefault.jpg,12654,122,9,NaN,xVaTzgcJttI,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,"[사랑, 하는, 너, 열차, 안, 에서, 함께, 노래, 듣고싶어, 애정, 싹트는, ...",1.0,0.0


In [83]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from konlpy.tag import Okt
import jpype

# 수동으로 JVM 경로 지정 (우중 노트북에서만 가능 -> 수정 필요!!)
jvmpath = "/Users/byeon-ujung/Library/Java/JavaVirtualMachines/openjdk-23/Contents/Home/lib/server/libjvm.dylib"
okt = Okt(jvmpath=jvmpath)


# 1. 데이터 로딩
path = r'postP_all_youtubeAPI_info.csv'
df = pd.read_csv(path)
df = df.dropna(subset=['title']).reset_index(drop=True)

# 2. 정제 함수
def clean_text(text):
    text = re.sub(r"[^\uAC00-\uD7A3a-zA-Z0-9\s]", "", str(text))
    return re.sub(r"\s+", " ", text).strip()

# 3. 형태소 분석기 + 불용어
okt = Okt()
stopwords = {'은', '는', '이', '가', '을', '와', '과', '를', '에', '의', '도', '다', '한', '그리고', '으로', '부터'}

preprocessed_titles = []
token_lists = []

for title in df['title']:
    cleaned = clean_text(title)
    tokens = okt.morphs(cleaned)
    tokens = [t for t in tokens if t not in stopwords]
    preprocessed_titles.append(" ".join(tokens))
    token_lists.append(tokens)

df['token_list'] = token_lists

# 4. 키워드 사전 정의

# 감성 키워드 
emotion_words = {
    '감성', '무드', '잔잔', '아늑', '몽환', '근사', '따뜻', '감미', '로맨틱', '인디', 'R&B','새벽', '사랑' , '혼자', '감미', '노을', '낭만',
    '여운', '포근', '섬세', '감동', '고요', '센치', '밤공기', '부드러운', '나른한', '그윽한', '온기' ,
    '멜로디', '여리여리', '감정선', '흘러가는', '그때 그 감성', '비', 
    '잊지 못할', '쓸쓸한', '낭만', '바람결', '피아노','분위기', '감미로운', '여백', '밤하늘', '창가', '조용한', '빗소리',
    '꿈같은', '설레는', '따스한', '흐림', '스미는', '그날의', '목소리', '공기'
}

# 일상 키워드
daily_words = {
    '일상', '아침', '저녁', '출근', '퇴근', '산책', '브런치', '도심', '하루', '일기', '편안', '여유',  '잔잔', '산뜻'
    '휴식', '오후', '오후3시', '자취', '혼밥', '책상', '창밖', '커피', '공원', '루틴', '나의 하루', '평온한',
    '나른한', '퇴근길', '혼자만의 시간', '작심삼일', '일요일', '카페', '째즈' , '재즈' ,'등교' , '하교',
    '루틴 음악', '아지트', '일상 회복', '무드등', '잔잔한', '루틴한', '창문', '햇살', '방안', '자명종', '출근길','퇴근길', '하늘', '조용한 시간',
    '텀블러', '점심시간', '피로', '혼자 듣는', '느긋한', '거실' ,'여운'
}

# 5. 자동 라벨링 (감성/일상 키워드 매칭)
def is_emotion(tokens):
    return 1 if any(word in tokens for word in emotion_words) else pd.NA  # 준지도니까 NA 허용
def is_daily(tokens):
    return 1 if any(word in tokens for word in daily_words) else pd.NA

df['감성'] = df['token_list'].apply(is_emotion)
df['일상'] = df['token_list'].apply(is_daily)

# # 1. 라벨이 있는 데이터만 필터링
# df_emotion_labeled = df[df['감성'].notna()].copy()
# df_daily_labeled = df[df['일상'].notna()].copy()

# # 2. 감성 예측용 TF-IDF
# vectorizer_emotion = TfidfVectorizer()
# X_emotion = vectorizer_emotion.fit_transform(df_emotion_labeled['token_list'].apply(lambda x: " ".join(x)))
# y_emotion = df_emotion_labeled['감성'].astype(int)

# # 3. 일상 예측용 TF-IDF
# vectorizer_daily = TfidfVectorizer()
# X_daily = vectorizer_daily.fit_transform(df_daily_labeled['token_list'].apply(lambda x: " ".join(x)))
# y_daily = df_daily_labeled['일상'].astype(int)

# # 4. 감성 모델 학습
# model_emotion = RandomForestClassifier()
# model_emotion.fit(X_emotion, y_emotion)

# # 5. 일상 모델 학습
# model_daily = RandomForestClassifier()
# model_daily.fit(X_daily, y_daily)

# # 6. 라벨이 없는 데이터 예측
# df_emotion_unlabeled = df[df['감성'].isna()]
# X_emotion_unlabeled = vectorizer_emotion.transform(df_emotion_unlabeled['token_list'].apply(lambda x: " ".join(x)))
# emotion_preds = model_emotion.predict(X_emotion_unlabeled)

# df.loc[df_emotion_unlabeled.index, '감성'] = emotion_preds

# df_daily_unlabeled = df[df['일상'].isna()]
# X_daily_unlabeled = vectorizer_daily.transform(df_daily_unlabeled['token_list'].apply(lambda x: " ".join(x)))
# daily_preds = model_daily.predict(X_daily_unlabeled)

# df.loc[df_daily_unlabeled.index, '일상'] = daily_preds
# df



In [84]:
df.loc[(df['감성']==1) | (df['일상']==1), : ]

,Unnamed: 0,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상
0,0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"[Playlist, 재즈, 로, 무드있, 고, 잔잔하게, 내, 방, 근사하게, 만드...",1,1
1,1,Jazzne | 기분Jazz네,287000,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,https://www.youtube.com/watch?v=kERWAGtltBU,2025-02-01T23:50:00Z,https://i.ytimg.com/vi/kERWAGtltBU/hqdefault.jpg,80929,404,22,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",kERWAGtltBU,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,"[Playlist, 일, 할, 때, 틀어, 놓기, 좋은, 업무, 용, 재즈, 집중,...",<NA>,1
2,2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"[Playlist, 여유롭게, 쉬, 고, 싶을, 때, 잔잔히, 틀기, 좋은, 아늑한...",<NA>,1
3,3,Jazzne | 기분Jazz네,287000,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,https://www.youtube.com/watch?v=CI331OcJ8Do,2025-01-18T23:50:00Z,https://i.ytimg.com/vi/CI331OcJ8Do/hqdefault.jpg,265932,1042,45,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",CI331OcJ8Do,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,"[Playlist, 기분, 좋은, 아침, 여, 모닝, 겨울, 재즈, 하루, 시작, ...",<NA>,1
4,4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","[Playlist, 재즈, 내리는, 계절, 겨울, 눈, 오는, 날, 듣기, 좋은, ...",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14331,14331,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,https://www.youtube.com/watch?v=fLNAk4NYOJU,2024-03-21T04:00:19Z,https://i.ytimg.com/vi/fLNAk4NYOJU/hqdefault.jpg,64905,790,27,NaN,fLNAk4NYOJU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,"[벚꽃, 만, 개, 하, 봄, 봄, 햇살, 함께, 듣기, 좋은, 제이팝, JPOP,...",<NA>,1
14333,14333,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,https://www.youtube.com/watch?v=Sh41rnVFlhU,2024-03-09T09:23:50Z,https://i.ytimg.com/vi/Sh41rnVFlhU/hqdefault.jpg,7418,98,18,NaN,Sh41rnVFlhU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,"[도쿄, 긴자, 밤, 닮은, 중독, 적, 인, 리듬, 멜로디, 힙, 사운드, 제이팝...",1,<NA>
14334,14334,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,https://www.youtube.com/watch?v=z1UGEkHIoCY,2024-02-26T10:00:40Z,https://i.ytimg.com/vi/z1UGEkHIoCY/hqdefault.jpg,16421,218,9,NaN,z1UGEkHIoCY,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,"[쓸쓸한, 도쿄, 퇴근길, 조용한, 밤, 듣기, 좋은, 제이팝, JPOP, 플레이,...",1,1
14335,14335,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,https://www.youtube.com/watch?v=xVaTzgcJttI,2024-02-22T04:03:09Z,https://i.ytimg.com/vi/xVaTzgcJttI/hqdefault.jpg,12654,122,9,NaN,xVaTzgcJttI,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,"[사랑, 하는, 너, 열차, 안, 에서, 함께, 노래, 듣고싶어, 애정, 싹트는, ...",1,<NA>


In [77]:
df_emotion_labeled

,Unnamed: 0,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상
0,0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"[Playlist, 재즈, 로, 무드있, 고, 잔잔하게, 내, 방, 근사하게, 만드...",1,1
4,4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","[Playlist, 재즈, 내리는, 계절, 겨울, 눈, 오는, 날, 듣기, 좋은, ...",1,1
5,5,Jazzne | 기분Jazz네,287000,Playlist | 적당히 신나는 템포의 편안한 겨울 감성 재즈 with 뉴욕🗽🇺🇸...,https://www.youtube.com/watch?v=xF_r6vfY9L4,2025-01-04T23:50:00Z,https://i.ytimg.com/vi/xF_r6vfY9L4/hqdefault.jpg,158522,789,29,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",xF_r6vfY9L4,Playlist | 적당히 신나는 템포의 편안한 겨울 감성 재즈 with 뉴욕🗽🇺🇸...,"[Playlist, 적당히, 신나는, 템포, 편안한, 겨울, 감성, 재즈, with...",1,1
6,6,Jazzne | 기분Jazz네,287000,"Playlist | 눈 내리는 겨울, 재즈와 함께 ❤️🎧 | 틀어놓고 편안히 듣는 ...",https://www.youtube.com/watch?v=QJuQNPje0Y4,2024-12-28T23:50:00Z,https://i.ytimg.com/vi/QJuQNPje0Y4/hqdefault.jpg,299456,1157,26,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",QJuQNPje0Y4,"Playlist | 눈 내리는 겨울, 재즈와 함께 ❤️🎧 | 틀어놓고 편안히 듣는 ...","[Playlist, 눈, 내리는, 겨울, 재즈, 함께, 틀어놓고, 편안히, 듣는, ...",1,1
7,7,Jazzne | 기분Jazz네,287000,"Playlist | 추운 겨울, 마음을 녹이는 따뜻한 감성 재즈..☕️❤️ | 일상...",https://www.youtube.com/watch?v=-rGGG0IdNh8,2024-12-21T23:50:00Z,https://i.ytimg.com/vi/-rGGG0IdNh8/hqdefault.jpg,112311,438,18,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",-rGGG0IdNh8,"Playlist | 추운 겨울, 마음을 녹이는 따뜻한 감성 재즈..☕️❤️ | 일상...","[Playlist, 추운, 겨울, 마음, 녹이는, 따뜻한, 감성, 재즈, 일상, 속...",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14327,14327,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 신주쿠 중앙공원에서 즐기는 싱그러운 시작의 멜로디🍃 ㅣ ...,https://www.youtube.com/watch?v=AbDJl2CTPLs,2024-04-23T01:23:38Z,https://i.ytimg.com/vi/AbDJl2CTPLs/hqdefault.jpg,39438,512,24,NaN,AbDJl2CTPLs,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 신주쿠 중앙공원에서 즐기는 싱그러운 시작의 멜로디🍃 ㅣ ...,"[신주쿠, 중앙, 공원, 에서, 즐기는, 싱그러운, 시작, 멜로디, 청량하고, 상쾌...",1,1
14333,14333,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,https://www.youtube.com/watch?v=Sh41rnVFlhU,2024-03-09T09:23:50Z,https://i.ytimg.com/vi/Sh41rnVFlhU/hqdefault.jpg,7418,98,18,NaN,Sh41rnVFlhU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 도쿄 긴자의 밤을 닮은 중독적인 리듬과 멜로디💊 ㅣ 힙한...,"[도쿄, 긴자, 밤, 닮은, 중독, 적, 인, 리듬, 멜로디, 힙, 사운드, 제이팝...",1,<NA>
14334,14334,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,https://www.youtube.com/watch?v=z1UGEkHIoCY,2024-02-26T10:00:40Z,https://i.ytimg.com/vi/z1UGEkHIoCY/hqdefault.jpg,16421,218,9,NaN,z1UGEkHIoCY,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,"[쓸쓸한, 도쿄, 퇴근길, 조용한, 밤, 듣기, 좋은, 제이팝, JPOP, 플레이,...",1,1
14335,14335,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,https://www.youtube.com/watch?v=xVaTzgcJttI,2024-02-22T04:03:09Z,https://i.ytimg.com/vi/xVaTzgcJttI/hqdefault.jpg,12654,122,9,NaN,xVaTzgcJttI,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,"[사랑, 하는, 너, 열차, 안, 에서, 함께, 노래, 듣고싶어, 애정, 싹트는, ...",1,<NA>


In [79]:
df_daily_labeled

,Unnamed: 0,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상
0,0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"[Playlist, 재즈, 로, 무드있, 고, 잔잔하게, 내, 방, 근사하게, 만드...",1,1
1,1,Jazzne | 기분Jazz네,287000,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,https://www.youtube.com/watch?v=kERWAGtltBU,2025-02-01T23:50:00Z,https://i.ytimg.com/vi/kERWAGtltBU/hqdefault.jpg,80929,404,22,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",kERWAGtltBU,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,"[Playlist, 일, 할, 때, 틀어, 놓기, 좋은, 업무, 용, 재즈, 집중,...",<NA>,1
2,2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"[Playlist, 여유롭게, 쉬, 고, 싶을, 때, 잔잔히, 틀기, 좋은, 아늑한...",<NA>,1
3,3,Jazzne | 기분Jazz네,287000,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,https://www.youtube.com/watch?v=CI331OcJ8Do,2025-01-18T23:50:00Z,https://i.ytimg.com/vi/CI331OcJ8Do/hqdefault.jpg,265932,1042,45,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",CI331OcJ8Do,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,"[Playlist, 기분, 좋은, 아침, 여, 모닝, 겨울, 재즈, 하루, 시작, ...",<NA>,1
4,4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","[Playlist, 재즈, 내리는, 계절, 겨울, 눈, 오는, 날, 듣기, 좋은, ...",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14327,14327,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 신주쿠 중앙공원에서 즐기는 싱그러운 시작의 멜로디🍃 ㅣ ...,https://www.youtube.com/watch?v=AbDJl2CTPLs,2024-04-23T01:23:38Z,https://i.ytimg.com/vi/AbDJl2CTPLs/hqdefault.jpg,39438,512,24,NaN,AbDJl2CTPLs,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 신주쿠 중앙공원에서 즐기는 싱그러운 시작의 멜로디🍃 ㅣ ...,"[신주쿠, 중앙, 공원, 에서, 즐기는, 싱그러운, 시작, 멜로디, 청량하고, 상쾌...",1,1
14330,14330,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 오키나와 잔탄 공원의 레스토랑에서 바다를 바라보며🌊ㅣ 여...,https://www.youtube.com/watch?v=b2CdWAFQkIQ,2024-03-28T03:30:12Z,https://i.ytimg.com/vi/b2CdWAFQkIQ/hqdefault.jpg,28175,336,27,NaN,b2CdWAFQkIQ,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 오키나와 잔탄 공원의 레스토랑에서 바다를 바라보며🌊ㅣ 여...,"[오키나와, 잔, 탄, 공원, 레스토랑, 에서, 바다, 바라보며, 여유, 느껴지는,...",<NA>,1
14331,14331,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,https://www.youtube.com/watch?v=fLNAk4NYOJU,2024-03-21T04:00:19Z,https://i.ytimg.com/vi/fLNAk4NYOJU/hqdefault.jpg,64905,790,27,NaN,fLNAk4NYOJU,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 벚꽃이 만개하는 봄🌸 ㅣ 봄 햇살과 함께 듣기 좋은 제이...,"[벚꽃, 만, 개, 하, 봄, 봄, 햇살, 함께, 듣기, 좋은, 제이팝, JPOP,...",<NA>,1
14334,14334,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,https://www.youtube.com/watch?v=z1UGEkHIoCY,2024-02-26T10:00:40Z,https://i.ytimg.com/vi/z1UGEkHIoCY/hqdefault.jpg,16421,218,9,NaN,z1UGEkHIoCY,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 쓸쓸한 도쿄의 퇴근길💼 ㅣ 조용한 밤에 듣기 좋은 제이팝...,"[쓸쓸한, 도쿄, 퇴근길, 조용한, 밤, 듣기, 좋은, 제이팝, JPOP, 플레이,...",1,1


In [8]:
df.loc[df['감성'] == 1,:]

,Unnamed: 0,channel_title,subscriber_count,title,video_url,publish_date,thumbnail_url,views,likes,comments,hashtags,video_id,title_normalized,token_list,감성,일상
0,0,Jazzne | 기분Jazz네,287000,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,https://www.youtube.com/watch?v=s6suc9Xz0rM,2025-02-08T23:50:00Z,https://i.ytimg.com/vi/s6suc9Xz0rM/hqdefault.jpg,22675,423,82,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",s6suc9Xz0rM,Playlist | 재즈로 무드있고 잔잔하게🌙 💟 | 내 방을 근사하게 만드는 로맨...,"[Playlist, 재즈, 로, 무드있, 고, 잔잔하게, 내, 방, 근사하게, 만드...",1,1
1,1,Jazzne | 기분Jazz네,287000,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,https://www.youtube.com/watch?v=kERWAGtltBU,2025-02-01T23:50:00Z,https://i.ytimg.com/vi/kERWAGtltBU/hqdefault.jpg,80929,404,22,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",kERWAGtltBU,Playlist | 일할 때 틀어 놓기 좋은 업무용 재즈 🖥️🎧 | 집중력을 높이는...,"[Playlist, 일, 할, 때, 틀어, 놓기, 좋은, 업무, 용, 재즈, 집중,...",1,1
2,2,Jazzne | 기분Jazz네,287000,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,https://www.youtube.com/watch?v=8VcAaEfuc0s,2025-01-25T23:50:00Z,https://i.ytimg.com/vi/8VcAaEfuc0s/hqdefault.jpg,61860,324,19,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",8VcAaEfuc0s,Playlist | 여유롭게 쉬고 싶을 때 잔잔히 틀기 좋은 아늑한 겨울 재즈 | ...,"[Playlist, 여유롭게, 쉬, 고, 싶을, 때, 잔잔히, 틀기, 좋은, 아늑한...",1,1
3,3,Jazzne | 기분Jazz네,287000,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,https://www.youtube.com/watch?v=CI331OcJ8Do,2025-01-18T23:50:00Z,https://i.ytimg.com/vi/CI331OcJ8Do/hqdefault.jpg,265932,1042,45,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",CI331OcJ8Do,Playlist | 기분 좋은 아침을 여는 모닝 겨울 재즈 ☀️🪟 | 하루를 시작하...,"[Playlist, 기분, 좋은, 아침, 여, 모닝, 겨울, 재즈, 하루, 시작, ...",1,1
4,4,Jazzne | 기분Jazz네,287000,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...",https://www.youtube.com/watch?v=AOrtPpF1t7U,2025-01-11T23:50:00Z,https://i.ytimg.com/vi/AOrtPpF1t7U/hqdefault.jpg,327122,1305,43,"기분째즈네, 기분재즈네, 재즈네, moodisjazz, 재즈피아노, 재즈란무엇인가,...",AOrtPpF1t7U,"Playlist | 재즈가 내리는 계절, 겨울 🌨️❣️ | 눈 오는 날 듣기 좋은 ...","[Playlist, 재즈, 내리는, 계절, 겨울, 눈, 오는, 날, 듣기, 좋은, ...",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14335,14335,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,https://www.youtube.com/watch?v=xVaTzgcJttI,2024-02-22T04:03:09Z,https://i.ytimg.com/vi/xVaTzgcJttI/hqdefault.jpg,12654,122,9,NaN,xVaTzgcJttI,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 사랑하는 너와 열차 안에서 함께 이 노래를 듣고싶어💝 ㅣ...,"[사랑, 하는, 너, 열차, 안, 에서, 함께, 노래, 듣고싶어, 애정, 싹트는, ...",1,1
14336,14336,마치다 machida,6590,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 비 오는 오후에 도쿄 메이지 신궁에서☔ ㅣ 비 오는 날에...,https://www.youtube.com/watch?v=WEkYWyi1gfA,2024-02-19T04:00:11Z,https://i.ytimg.com/vi/WEkYWyi1gfA/hqdefault.jpg,19310,245,15,NaN,WEkYWyi1gfA,𝗝-𝗣𝗼𝗽 𝗣𝗹𝗮𝘆𝗹𝗶𝘀𝘁 비 오는 오후에 도쿄 메이지 신궁에서☔ ㅣ 비 오는 날에...,"[비, 오는, 오후, 도쿄, 메이지, 신궁, 에서, 비, 오는, 날, 듣기, 좋은,...",1,1
14337,14337,볼우물 DIMPLE,18800,[𝐒𝐕𝐓 𝐏𝐋𝐀𝐘𝐋𝐈𝐒𝐓] 나를 토닥토닥 위로해주는 세븐틴 노래 모음⋆˚ ☁︎ ˚⋆...,https://www.youtube.com/watch?v=ilDE-AjDh8Q,2024-06-21T12:00:47Z,https://i.ytimg.com/vi/ilDE-AjDh8Q/hqdefault.jpg,57744,1438,49,NaN,ilDE-AjDh8Q,[𝐒𝐕𝐓 𝐏𝐋𝐀𝐘𝐋𝐈𝐒𝐓] 나를 토닥토닥 위로해주는 세븐틴 노래 모음⋆˚ ☁︎ ˚⋆...,"[나를, 토닥토닥, 위로, 해주는, 세븐, 틴, 노래, 모음, 4, 편, 커버, 곡...",1,1
14338,14338,볼우물 DIMPLE,18800,[𝐒𝐕𝐓 𝐏𝐋𝐀𝐘𝐋𝐈𝐒𝐓] 나를 토닥토닥 위로해주는 세븐틴 노래 모음⋆˚ ☁︎ ˚⋆...,https://www.youtube.com/watch?v=s32rRIsfiFI,2024-06-04T12:00:52Z,https://i.ytimg.com/vi/s32rRIsfiFI/hqdefault.jpg,10627,287,11,NaN,s32rRIsfiFI,[𝐒𝐕𝐓 𝐏𝐋𝐀𝐘𝐋𝐈𝐒𝐓] 나를 토닥토닥 위로해주는 세븐틴 노래 모음⋆˚ ☁︎ ˚⋆...,"[나를, 토닥토닥, 위로, 해주는, 세븐, 틴, 노래, 모음, 4, 편, 커버, 곡...",1,1
